# Notebook for analysing imaging data (in vitro and ex vivo) 

This use this jupyter notebook for the analysis of your in vitro imaging data (best suited for cell culture data). Before you run this notebook there are a few steps you need to do first:

1. Load your raw imaging tiffs into imagej/fiji
2. Draw the ROIs **IMPORTANT**: Last ROI must be a background ROI 
3. Calculate mean grey value with imagej/fiji 
4. Export your results as csv file



In [ ]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


# Pre processing function:
def pre_processing(df):
    background=df[df.columns[-1]].tolist()  #select background column and make list
    df=df.drop(df.columns[-1], axis=1) #delete last column (background)
    df=df.drop(df.columns[0], axis=1) #delte first column (frame)
    df_background_sub=df.sub(background, axis=0) #subtract background from every other column
    
    return df_background_sub

def dff_calc(df_input,f_base_value=0,dff_median=False):
    
    for column_name in df_input:
        column=df_input[column_name]
        column=column.astype('float')
        if dff_median:
            median_value=column.median()
            dff_series=column.apply((lambda x: (x - median_value)/median_value))
            df_input[column_name]=dff_series
        else:
            dff_series=column.apply((lambda x: (x - column.iloc[f_base_value])/column.iloc[f_base_value])) 
            df_input[column_name]=dff_series
        
    return df_input




## Defining some parameters for your analysis

In the next cell define some parameter:

* Do you want to perform preprocessing (i.e. background substraction) of your imagej/fiji data? 
 * default: `pre_processing_bool=True` <br>
<br>
* Do you want to exclude specific ROIs?         
 * default: `exclude_rois=True` <br>
<br>    
* Do you want to convert frames to seconds? If so input framerate in fps.
 * default: `frames_to_seconds=True` 
 * default: `fps_rate=1` <br>
<br>

**Next**: Input the path to your results csv file from imagej/fiji input the variable: path_to_data

In [ ]:
pre_processing_bool=True
frames_to_seconds=True
fps_rate=5 

#path_to_data='D:/Lutz imaging/Gq-Pathway Sensor/20210513/Dish3/Analysis/Results_20210513_dish3'
path_to_data='D:/Lutz imaging/23082021/Slice1/Results'

file_extension='.csv'

if pre_processing_bool==True:
    df=pre_processing(pd.read_csv(path_to_data + file_extension))
else:
    df=pd.read_csv(path_to_data + file_extension)

if frames_to_seconds:
    df.index=df.index/fps_rate
    x_label='Time (s)'
else:
    x_label='Frames'


## Calculation of $\Delta$f/f

Now you can calculate $\Delta$f/f but first define the parameters for that: 
* If you want to calculate $\Delta$f/f with the first frame as your f0, you do not need to change anything.
* If you want to calculate $\Delta$f/f with the median fluorescence as your f0, set the parameter `dff_median=True` <br> (i.e. `dff_frame=dff_calc(df_copy, dff_median=True)`)

It also plots each of your ROIs in a subplot and also all ROIs combined
    



In [ ]:
df_copy=df.copy()
dff_frame=dff_calc(df_copy, dff_median=True)     

In [ ]:
#plotting individual ROIs in individual subplots
col_num=len(dff_frame.columns)
number_of_subplots=list(range(0,len(dff_frame.columns)))
axes=dff_frame.plot(subplots=True, layout=(7, 5), figsize=(24, 24), sharex=True,sharey=True, legend=None, title=number_of_subplots)

for c in axes:
   for ax in c:
      ax.axhline(y=0, color='k', ls='dashed')

plt.show()

#plotting individual ROIs in ONE plot (overlayed)
dff_frame.plot(legend=None)
ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')
plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.show()

## Excluding ROIs (optional)

If you want exclude some of the ROIs from above set `exclude_rois=True` (default: `exclude_rois=False`) and put the numbers of the ROIs you want to exclude into the list and seperate every ROI by a "," (`list_of_excluded_rois`) below. **IMPORTANT**: Start counting from 0!

It also saves the a new csv file into the current path with the addtion of *_cleaned*

In [ ]:
exclude_rois=False
list_of_excluded_rois=[1,4]

if exclude_rois:
    cleaned_df=dff_frame.copy()
    exclude_columns=cleaned_df.iloc[:, list_of_excluded_rois].columns
    cleaned_df=cleaned_df.drop(columns=exclude_columns)    
    cleaned_df.to_csv((path_to_data + '_cleaned' + file_extension), index=False)  
    
    df_mean=cleaned_df.mean(axis=1)
    df_std=cleaned_df.std(axis=1)
    df_sem=cleaned_df.sem(axis=1)
    
else:
    cleaned_df=dff_frame.copy()
    cleaned_df.to_csv((path_to_data + '_cleaned' + file_extension), index=False)
    
    df_mean=dff_frame.mean(axis=1)
    df_std=dff_frame.std(axis=1)
    df_sem=dff_frame.sem(axis=1)

Now plot the ROIs again to make sure you deleted the correct ROIs

In [ ]:
#plotting individual ROIs in individual subplots
if exclude_rois:
    
    col_num=len(cleaned_df.columns)
    number_of_subplots=list(range(0,len(cleaned_df.columns)))
    axes=cleaned_df.plot(subplots=True, layout=(7, 5), figsize=(24, 24), sharex=True,sharey=True, legend=None, title=number_of_subplots)

    for c in axes:
       for ax in c:
          ax.axhline(y=0, color='k', ls='dashed')

    plt.show()

    #plotting individual ROIs in ONE plot (overlayed)
    cleaned_df.plot(legend=None)
    ax = plt.gca()
    ax.axhline(y=0, color='k', ls='dashed')
    plt.xlabel(x_label)
    plt.ylabel('\u0394 F/F')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.show()

## Plotting of the mean time courses

Now you can start plotting the mean of all your ROIs. But first define if you want to save the images (default: `save_images=False`)

In [ ]:
save_images=False

#plotting mean + SD

plt.plot(df_mean, color='black', linewidth=0.75)
plt.fill_between(df_mean.index, df_mean-df_std, df_mean+df_std, alpha=0.5)

ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
plt.title('Mean + SD')
if save_images:
    plt.savefig((path_to_data[:-4] + '_mean_SD' + '.svg'), format = 'svg', dpi=300) # uncommand this if you want to save it as vector graphic
plt.show()


In [ ]:
#plotting mean + SEM

plt.plot(df_mean, color='black', linewidth=0.75)
plt.fill_between(df_mean.index, df_mean-df_sem, df_mean+df_sem, alpha=0.5)

ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
plt.title('Mean + SEM')
if save_images:
    plt.savefig((path_to_data[:-4] + '_mean_SEM' + '.svg'), format = 'svg', dpi=300) # uncommand this if you want to save it as vector graphic
plt.show()

## Analysis of all ROIs from multiple dishes/trials

Now you can start analyzing all ROIs from several dishes or trials together. Just put the path to your new csv files (you calculated and exported them in the previous single dish analysis) into the list `trial_list` in the cell below. Example:

`trial_list=[(pd.read_csv('D:/Lutz imaging/Gq-Pathway Sensor/20210519/Dish1/Analysis/Results_20210519_dish1_new.csv')),
            (pd.read_csv('D:/Lutz imaging/Gq-Pathway Sensor/20210519/Dish3/Analysis/Results_20210519_dish3_new.csv')),
            (pd.read_csv('D:/Lutz imaging/Gq-Pathway Sensor/20210519/Dish4/Analysis/Results_20210519_dish4_new.csv')),
            (pd.read_csv('D:/Lutz imaging/Gq-Pathway Sensor/20210519/Dish5/Analysis/Results_20210519_dish5_new.csv')),
            (pd.read_csv('D:/Lutz imaging/Gq-Pathway Sensor/20210521/Dish1/Analysis/Results_20210521_dish1_new.csv')),
            (pd.read_csv('D:/Lutz imaging/Gq-Pathway Sensor/20210521/Dish3/Analysis/Results_20210521_dish3_new.csv'))
            ]`
            
**IMPORTANT**: The single dish analysis puts an *_cleaned* at the end of your path, so don't forget to include this here.




**IMPORTANT**: If you just want to run multi dish analysis, run the next cell. Otherwise you can ignore it. 

In [ ]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
trial_list=[(pd.read_csv('D:/Lutz imaging/23082021/Slice1/Results_cleaned.csv')) 
            ]

col_num=0
for it_count, number in enumerate(trial_list):
    col_num=col_num + len(number.columns)
    number.columns=list(number.columns + '_Trial_' + str(it_count+1))
    
combined_df=pd.concat(trial_list, axis=1) #combine all trials in one dataframe

combined_df.reset_index(drop=True, inplace=True)

#combined_df


**Important**: Because you reseted the DataFrame Index in the cell before, you have to chose again if you want to convert frames into seconds. Also define the `fps_rate`  again

In [ ]:
frames_to_seconds=True
fps_rate=5

if frames_to_seconds:
    combined_df.index=combined_df.index/fps_rate
    x_label='Time (s)'
else:
    x_label='Frames'

## Optional calculation of $\Delta$f/f

In the single dish/single trial analysis, you saved the data **AFTER** you calculated $\Delta$f/f. Therefore, if you run the notebook completely there is no need to calculated $\Delta$f/f. 

However, there is an option to do it at this point. Just set `run_dff_calc_again=True` (default:`run_dff_calc_again=False`)

Now you can calculate $\Delta$f/f but first define the parameters for that: 
* If you want to calculate $\Delta$f/f with the first frame as your f0, you do not need to change anything.
* If you want to calculate $\Delta$f/f with the median fluorescence as your f0, set the parameter `dff_median=True` <br> (i.e. `dff_frame=dff_calc(df_copy, dff_median=True)`)
    

In [ ]:
run_dff_calc_again=False

if run_dff_calc_again:
    combined_df_copy=combined_df.copy()
    dff_combined_df=dff_calc(combined_df_copy, dff_median=True)
else:
    dff_combined_df=combined_df.copy()
    combined_df_mean=dff_combined_df.mean(axis=1)
    combined_df_std=dff_combined_df.std(axis=1)
    combined_df_sem=dff_combined_df.sem(axis=1)

## Ploting of the combined data

Now, that you analysed and combined all data from all trials/dishes into one DataFrame you can start plotting. 


You have some options here:

* Here you have the option to also plot your wash in times (if you have any). Just set `plot_wash_in_1=True` and/or `plot_wash_in_2=True`. Also, if you want to do that, make sure define the timepoints (start and end points).<br> **IMPORTANT**: If you chose the option to convert frames to seconds you also have to set the timepoints as seconds and not as frames.<br>
<br>
* You can also decide to save the plot. Just set `save_images=True` (default: `save_images=False`). Also define the path and the the file name (withou file extension). Example: `path_to_save_date='E:/Lutz imaging/Gq-Pathway Sensor/Plots/KD_mut/Results_20210521_dish3_new'`



**Mean +/- SD**


In [ ]:
save_images=False
path_to_save_date='E:/Lutz imaging/Gq-Pathway Sensor/Plots/KD_mut/Results_20210521_dish3_new'

plot_wash_in_1=False

start_wash_in_1=600 #add start frame of first wash in
end_wash_in_1=700 #add end frame of first wash in

if frames_to_seconds:
    start_wash_in_1=start_wash_in_1/fps_rate
    end_wash_in_1=end_wash_in_1/fps_rate


plot_wash_in_2=False

start_wash_in_2=1000 #add start frame of second wash in
end_wash_in_2=1100 #add end frame of second wash in

if frames_to_seconds:
    start_wash_in_2=start_wash_in_2/fps_rate
    end_wash_in_2=end_wash_in_2/fps_rate


In [ ]:
text_for_fig= 'n=' + str(col_num) + ' cells'

#plot mean + SD
plt.figure(figsize=(10,5))
if plot_wash_in_1:
    plt.axvspan(start_wash_in_1, end_wash_in_1, color='grey', alpha=0.3, lw=0)
if plot_wash_in_2:
    plt.axvspan(start_wash_in_2, end_wash_in_2, color='grey', alpha=0.3, lw=0)
plt.plot(combined_df_mean, c='black')
plt.fill_between(combined_df_mean.index, combined_df_mean-combined_df_std, combined_df_mean+combined_df_std, alpha=0.5)

ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
plt.title('Mean + SD')

plt.text(0.9, 0.9,text_for_fig,
      horizontalalignment='center',
      verticalalignment='center',
      transform = plt.gca().transAxes)
if save_images:
    plt.savefig((path_to_save_date + 'all_trials_mean_sd.svg'), format='svg', dpi=300)
plt.show()


Here you can plot **Mean +/- SEM**

In [ ]:
plt.figure(figsize=(10,5))

if plot_wash_in_1:
    plt.axvspan(start_wash_in_1, end_wash_in_1, color='grey', alpha=0.3, lw=0)
if plot_wash_in_2:
    plt.axvspan(start_wash_in_2, end_wash_in_2, color='grey', alpha=0.3, lw=0)
    
plt.plot(combined_df_mean, c='black')
plt.fill_between(combined_df_mean.index, combined_df_mean-combined_df_sem, combined_df_mean+combined_df_sem, alpha=0.5)

ax = plt.gca()
ax.axhline(y=0, color='k', ls='dashed')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.xlabel(x_label)
plt.ylabel('\u0394 F/F')
plt.title('Mean + SEM')
plt.text(0.9, 0.9,text_for_fig,
      horizontalalignment='center',
      verticalalignment='center',
      transform = plt.gca().transAxes)

if save_images:
    plt.savefig((path_to_save_date + 'all_trials_mean_sem.svg'), format='svg', dpi=300)
plt.show()

Here you can plot the **heatmap**

In [ ]:

if frames_to_seconds:
    mesh_df= combined_df.copy()
    mesh_df.reset_index(drop=True, inplace=True)
    new_index=list(mesh_df.index)
    new_index.append(len(list(mesh_df.index)))
    new_index=pd.Series(new_index)/fps_rate
    new_columns=pd.Series(list(range(0, len(mesh_df.columns)+1)))
        
    fig, ax0 = plt.subplots(figsize=(10,8))
    im = ax0.pcolormesh(new_index,new_columns, (mesh_df.transpose()))
    fig.colorbar(im, ax=ax0, label='\u0394 F/F')
    ax0.set_xlabel('Time (s)')
    ax0.set_ylabel('ROI Number')   
    if save_images:
        plt.savefig((path_to_save_date + '_heatmap.svg'), format='svg', dpi=300)
    plt.show()

        
else:
    fig, ax0 = plt.subplots(figsize=(10,8))
    im = ax0.pcolormesh((combined_df.transpose()))
    fig.colorbar(im, ax=ax0, label='\u0394 F/F')

    ax0.set_xlabel('Frame')
    ax0.set_ylabel('ROI Number')

    ax0.axvline(x=start_wash_in_1, color='k', ls='dashed', alpha=0.75)
    ax0.axvline(x=start_wash_in_2, color='k', ls='dashed', alpha=0.75)
    if save_images:
        plt.savefig((path_to_save_date + '_heatmap.svg'), format='svg', dpi=300)   
    plt.show()